# Spark 

#### Purpose: 
To pull quickly pull out data from excel sheets without the hassle of hunting it yourself. 

#### Instructions: 
1. Place files into local drives (OneDrive works as well). **Note**: Make sure there is only 1. 
2. Input Spark ID. 
3. Click `Cell` and `Run All`

#### Future: 
Calculate the data and push out full dataframe and create a heatmap for quick and easy visual. 

In [3]:
# input name - File must be local or in GH One Drive
SparkID = "2564_kk10"

In [4]:
import pandas as pd
import os

# find file within local directories
filepath = "/Users/"

for root, dirs, files in os.walk(filepath):
    for file in files:
        if (SparkID + ".xlsx") in file:
            sparkFile = (os.path.join(root, file))

spark_data = pd.read_excel(sparkFile)


# function to pull out 260/320 abs
def colAbs(raw_spark):
    index260 = []
    index320 = []
    wells = []
    for names in raw_spark.iloc[35:(35 + 48), 0]:
        wells.append(names)
    for n260 in raw_spark.iloc[146:(146 + 48), 1]:
        index260.append(n260)
    for n320 in raw_spark.iloc[368:(368 + 48), 1]:
        index320.append(n320)
    # create one dataframe
    df = {"Wells": wells,
                 "abs260": index260,
                 "abs320": index320}
    df = pd.DataFrame(df)
    df = df.set_index("Wells")
    print(df)
    
colAbs(spark_data)

       abs260  abs320
Wells                
A1     0.1224  0.0442
B1     0.1228  0.0447
C1     0.1235  0.0440
D1     0.1253  0.0461
E1     0.1234  0.0452
F1     0.1247  0.0438
G1     0.1230  0.0446
H1     0.1260  0.0449
A2     0.1264  0.0446
B2     0.1236  0.0443
C2     0.1270  0.0447
D2     0.1251  0.0445
E2     0.1231  0.0437
F2     0.1247  0.0447
G2     0.1246  0.0441
H2     0.1304  0.0447
A3     0.1260  0.0443
B3     0.1391  0.0541
C3     0.1252  0.0444
D3     0.1188  0.0438
E3     0.1259  0.0441
F3     0.1228  0.0438
G3     0.1264  0.0442
H3     0.1309  0.0488
A4     0.1231  0.0444
B4     0.1313  0.0490
C4     0.1250  0.0448
D4     0.1239  0.0441
E4     0.1273  0.0447
F4     0.1233  0.0438
G4     0.1265  0.0440
H4     0.1262  0.0442
A5     0.1264  0.0448
B5     0.1243  0.0438
C5     0.1314  0.0478
D5     0.1250  0.0439
E5     0.1237  0.0438
F5     0.1281  0.0446
G5     0.1275  0.0442
H5     0.1291  0.0438
A6     0.1245  0.0443
B6     0.1257  0.0444
C6     0.1349  0.0485
D6     0.1